A list consisting of 7 green elements with LB=0.92773
was "updated to 11 elements" by red elements with LB=0.92774. The voting procedure was launched. On the very first iteration, it showed a "gap" of LB=0.92775. This result gave us an idea: at each new iteration of the voting procedure, replace the "weaker" (green) element with a newly obtained one, with an estimated LB=0.92775, assigning it a different color—for example, blue. We tried three iterations manually: the second iteration showed a slightly better result. The third iteration showed a slight weakening. Therefore, we will repeat this entire process not manually, but semi-automatically. For example, we will limit ourselves to seven iterations—the same number as our initial green elements. We will see what happens. If this enterprise is of any use, we will put the whole thing on automatic!

In [1]:
import numpy as np
import pandas as pd

import os,ast,shutil,copy

from bokeh.plotting import figure, gridplot 
from bokeh.io import output_file, show, output_notebook
output_notebook()


def bokeh_show(
        params,
        df_cross,
        show_figures1, 
        show_figures2, wps_fig2,
        color_cross):

    colors = [subm['color'] for subm in params['subm']]
    
    def dossier(js,subms,cols):
        def quant(i,js,subms,cols):
            return {"c" : i, "q" : sum([1 for subm in cols[i] if subm == subms[js]])}
        return {
            'name' : subms[js],
            'q_in' : [quant(i,js,subms,cols) for i in range(len(subms))]
        }
    alls = pd.read_csv(f'tida_desc.csv')
    matrix = [ast.literal_eval(str(row.alls)) for row in alls.itertuples()]
    subms = sorted(matrix[0])
    cols = [[data[i] for data in matrix] for i in range(len(subms))]
    df_subms = pd.DataFrame({f'col_{i}': [x[i] for x in matrix] for i in range(len(subms))})
    dossiers = [dossier(js,subms,cols) for js in range(len(subms))]
    subm_names = [one_dossier['name'] for one_dossier in dossiers]
    figures1,qss,i = [],[],0
    height = 85 if len(colors)==2\
        else 134 if len(colors)==3 else (154 if len(colors)==4 else 174)
    for one_dossier in dossiers: 
        i_col = 'alls. ' + str(one_dossier['q_in'][i]['c'])
        qs = [one['q'] for one in one_dossier['q_in']]
        x_names = [name.replace("Group","").replace("subm_","") for name in subm_names]
        width = 157  if len(colors) == 5\
            else (121 if len(colors) == 8\
            else (131 if len(colors) == 9\
            else (141 if len(colors) == 10\
            else (171 if len(colors) == 11 else 133))))
        f = figure(x_range=x_names,width=width, height=height, title=i_col)
        f.vbar(x=x_names, width=0.585, top=qs, color=colors)
        figures1.append(f)
        qss.append(qs)
        i+=1
    grid = gridplot([figures1])
    output_file('tida_alls.html')
    if show_figures1 == True: show(grid)
    sub_wts = params['subwts']
    main_wts = [subm['weight'] for subm in params['subm']]
    mms,acc_mass = [],[]
    for j in range(len(dossiers)):
        one_dossier = dossiers[j]
        qs = [one['q'] for one in one_dossier['q_in']]
        mm = [qs[h] * (main_wts[j] + sub_wts[h]) for h in range(len(sub_wts))]
        mass = sum(mm)
        mms.append(mm)
        acc_mass.append(round(mass))                        #subm_names[::-1]
    y_names = [name + " - " + str(mass) for name,mass in zip(subm_names,acc_mass)]
    f1 = figure(y_range=y_names, width=313, height=height, title='relations of general masses')
    f1.hbar(y=y_names, height=0.585, right=acc_mass, left=0, color=colors)
    output_file('tida_alls2.html')
    alls = [f'alls.{i}' for i in range(len(dossiers))]
    subm = [f'sub{i}'   for i in range(len(dossiers))] 
    mmsT  = np.asarray(mms).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = mmsT[i,:]
    f2 = figure(y_range=alls, height=height, width=274, title=" ( relations of columns masses )")
    f2.hbar_stack(subm, y='cols', height=0.585, color=colors, source=data)
    qssT  = np.asarray(qss).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = qssT[i,:]
    f3 = figure(y_range=alls, height=height, width=210, title="ratios in columns")
    f3.hbar_stack(subm, y='cols', height=0.585, color=colors, source=data)
    grid = gridplot([[f3,f2,f1]])
    show(grid)
    if show_figures2 == True:
        def read(params,i):
            FiN = params["path"] + params["subm"][i]["name"] + ".csv"
            target_name_back = {'target':params["target"],'pred':params["target"]}
            return pd.read_csv(FiN).rename(columns=target_name_back)
        dfs = [read(params,i) for i in range(len(params["subm"]))] + [df_cross]
        _height = 358 if len(params["subm"]) == 11 else 254
        f   = figure(width=785, height=_height)
        f.title.text = 'Click on legend entries to mute the corresponding lines'
        b,e        = 21000,21121
        line_x     = [dfs[i][b:e]['id']             for i in range(len(dfs))]
        line_y     = [dfs[i][b:e]['loan_paid_back'] for i in range(len(dfs))]
        color      = colors + [color_cross]
        alpha      = [0.8 for i in range(len(dfs)-1)] + [0.95]
        lws        = [1.0 for i in range(len(dfs)-1)] + [1.00]
        legend = subm_names + ['cross']
        for i in range(len(legend)):
            f.line(line_x[i], line_y[i], line_width=lws[i], color=color[i], alpha=alpha[i],
                   muted_color='white',legend_label=legend[i])
        f.legend.location = "top_left"
        f.legend.click_policy="mute"
        show(f)


def matrix_vs(path,fs_names):
    def load(path,fs_names):
        dfs = [pd.read_csv(path + name_subm +'.csv') for name_subm in fs_names]
        for i in range(len(dfs)):
            dfs[i] = dfs[i].rename(columns={"loan_paid_back": f'{fs_names[i]}'})
        dfsm = pd.merge(dfs[0], dfs[1], on="id")
        for i in range(2,len(dfs)):
            dfsm = pd.merge(dfsm,dfs[i],on='id')
        return dfsm   
    def make_list_vs(fs_names):
        list = []
        for i in range(0,len(fs_names)-1):
            for j in range(i+1,len(fs_names)):
                list.append(fs_names[i] + "_vs_" + fs_names[j])
        return list
    def get_mvs(dfs, list_vs):
        def get_abs_distance(x,t1,t2):
            return abs(x[t1]-x[t2])
        for vs in list_vs:
            t = vs.split('_vs_')
            dfs[vs] = dfs.apply(lambda x: get_abs_distance(x,t[0],t[1]), axis=1)
        return dfs   
    def distance_vs(name, st_names, list_vs, dfs):
        distances = []
        for st in st_names:
            vs_between = name + "_vs_" + st
            if vs_between not in list_vs:
                distances.append(0)
            else: distances.append(round(dfs[vs_between].sum()))
        return distances
    dfs = load(path,fs_names)
    list_vs = make_list_vs(fs_names)
    mvs = get_mvs(dfs, list_vs)
    m1 = pd.DataFrame({'subm':fs_names})
    m2 = pd.DataFrame({ name :distance_vs(name, fs_names, list_vs, mvs) for name in fs_names})
    matrix = pd.concat([m1,m2],axis=1)
    return matrix


def display_distances(params):
    files = [subm['name'] for subm in params['subm']]
    distances = matrix_vs ( params['path'], files )            
    display(distances)


def arr_colors(color):
    sg = ['silver','gainsboro']
    if color=='silver'or color=='S': return ['gray','silver','gold']          + sg
    if color=='red'   or color=='R': return ['darkorchid','orangered','crimson'] + sg
    if color=='green' or color=='G': return ['darkorchid','limegreen', 'darkgreen'] + sg
    if color=='blue'  or color=='B': return ['darkorchid','mediumblue','midnightblue'] + sg
    return ['black','dimgray','gray'] + sg


def convert(schema):
    colors = arr_colors(schema[2])
    dicts  = [
        {'name': schema[0][i],'weight':schema[1][i],'color':colors[i]} 
        for i in range(len(schema[0]))
    ]
    return {'subm':dicts}


def h_blend(
        params, _update={},
        cross='silver',
        details=False,
        fig1=False, fig2=False, wf2=555, 
        dtls=False, dist=False, subm=''):

    if isinstance(params, list): params = convert(params)

    if 'path' in _update: params.update(_update)
    
    color_cross, dk  = cross, copy.deepcopy(params)

    if details == True:
        dist = True
        show_details,show_figures1,show_figures2 = True,True,True
    else:
        show_details,show_figures1,show_figures2 = dtls,fig1,fig2
        
    file_short_names = [subm['name'] for subm in params['subm']]
    type_sort    = params['type_sort'][0]
    dk['asc']    = params['type_sort'][1]
    dk['desc']   = params['type_sort'][2]
    dk['id']     = params['id_target'][0]
    dk['target'] = params['id_target'][1]
# ------------------------------------------------------------------------
    def read(dk,i):
        tnm = dk["subm"][i]["name"]
        FiN = dk["path"] + tnm + ".csv"
        return pd.read_csv(FiN).rename(columns={
            'target':tnm, 'pred':tnm, dk["target"]:tnm})
        
    def merge(dfs_subm):
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=[dk['id']])
        for i in range(2, len(dk["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=[dk['id']])
        return df_subms
        
    def da(dk,sorting_direction,show_details):
        
        df_subms = merge([read(dk,i) for i in range(len(dk["subm"]))])
        cols = [col for col in df_subms.columns if col != dk['id']]
        short_name_cols = [c for c in cols]
        
        def alls1(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [t[0] for t in sorted(tes,key=lambda k:k[1],reverse=reverse)]
            return subms_sorted

        import random

        def alls2(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_random = [t[0] for t in tes]
            random.shuffle(subms_random)
            return subms_random

        alls = alls1 if type_sort == 'asc/desc' else alls2
            
        def summa(x,cs,wts,ic_alls): 
            return sum([x[cs[j]] * (wts[0][j] + wts[1][ic_alls[j]]) for j in range(len(cs))])
            
        wts = [[[e['weight'] for e in dk["subm"]], [w for w in dk["subwts" ]]]]
          
        def correct(x, cs=cols, wts=wts):
            i = [x['alls'].index(c) for c in short_name_cols]
            return summa(x,cs,wts[0],i)

        if len(wts) == 1:
            correct_sub_weights = [wt for wt in dk["subwts"]]
            weights = [subm['weight'] for subm in dk["subm"]]
            def correct(x, cs=cols, w=weights, cw=correct_sub_weights):
                ic = [x['alls'].index(c) for c in short_name_cols]
                cS = [x[cols[j]] * (w[j] + cw[ic[j]]) for j in range(len(cols))]
                return sum(cS)
                   
        def amxm(x, cs=cols):
            list_values = x[cs].to_list()
            mxm = abs(max(list_values)-min(list_values))
            return mxm

        if len(wts) > 1:
            df_subms['mx-m']   = df_subms.apply(lambda x: amxm   (x), axis=1)
        df_subms['alls']       = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[dk["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={dk["target"]:"ensemble"})
        df_subms.insert(loc=1, column=' _ ', value=['   '] * len(df_subms))
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.5f}'.format)
        vcols = [dk['id']]+[' _ '] + short_name_cols + [' _ ']+['alls']+[' _ ']+['ensemble']
        if len(wts) > 1: vcols.append([' _ '] + ['mx-m'])
        df_subms = df_subms[vcols]
        if show_details and sorting_direction=='desc': display(df_subms.head(5))
        pd.set_option('display.float_format', '{:.5f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":dk["target"]})
        if sorting_direction=='desc': 
            df_subms.to_csv(f'tida_{sorting_direction}.csv', index=False)
        return df_subms[[dk['id'],dk['target']]]
   
    def ensemble_da(dk,        show_details): 
        dfD    = da(dk,'desc', show_details)
        dfA    = da(dk,'asc',  show_details)
        dfA[dk['target']] = dk['desc']*dfD[dk['target']] + dfA[dk['target']]*dk['asc']
        return dfA

    da = ensemble_da(dk,show_details)
    bokeh_show(dk, da, show_figures1, show_figures2, wf2, color_cross)
    if dist == True: display_distances(params)
    if subm != '': da.to_csv(subm, index=False)
    return  da


def voting(rem_left=0,rem_right=1):
    print(f'\nVoting:\n')
    df_Top = pd.read_csv('/kaggle/input/24-november-2025-ps-s5e11/774.1.csv')
    target = 'loan_paid_back'
    tida_desc = pd.read_csv('/kaggle/working/tida_desc.csv')
    df_soluts = tida_desc[tida_desc.columns[2:13]]
    display(df_soluts)
    qnt_Majority_votes = df_soluts.shape[1] - rem_right
    df_vote = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
    np_soluts_sorted = np.sort(df_soluts.to_numpy(), axis=1)
    df_vote[target] = np.mean(np_soluts_sorted[:,rem_left:qnt_Majority_votes], axis=1)
    df_vote[target] = df_Top[target] *0.05 + 0.95* df_vote[target]
    print(f'\nComparison: df_Top _x  vs  df_vote _y\n')
    df_comparison = pd.merge(df_Top, df_vote, on='id')
    df_comparison["delta_back's"] = df_Top[target] - df_vote[target]
    display  (df_comparison)
    print(sum(df_comparison["delta_back's"]),
              df_comparison["delta_back's"][df_comparison["delta_back's"] < 0].sum())
    return df_vote

Loading BokehJS ...

## Copy files from dataset

In [2]:
new_path  = '/kaggle/working/public'

ds_path24 = '/kaggle/input/24-november-2025-ps-s5e11'

if os.path.isdir(new_path): shutil.rmtree(new_path); 
    
os.mkdir(new_path)

shutil.copy(ds_path24 +'/773.1.csv', new_path +'/773.1.csv')
shutil.copy(ds_path24 +'/773.5.csv', new_path +'/773.5.csv')
shutil.copy(ds_path24 +'/773.6.csv', new_path +'/773.6.csv')
shutil.copy(ds_path24 +'/773.8.csv', new_path +'/773.8.csv')
shutil.copy(ds_path24 +'/773.9.csv', new_path +'/773.9.csv')
shutil.copy(ds_path24 +'/773.10.csv',new_path +'/773.10.csv')
shutil.copy(ds_path24 +'/773.12.csv',new_path +'/773.12.csv')
shutil.copy(ds_path24 +'/774.1.csv', new_path +'/774.1.csv')
shutil.copy(ds_path24 +'/774.3.csv', new_path +'/774.3.csv')
shutil.copy(ds_path24 +'/774.4.csv', new_path +'/774.4.csv')
shutil.copy(ds_path24 +'/774.5.csv', new_path +'/774.5.csv')

'/kaggle/working/public/774.5.csv'

## Iteration_1

In [3]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         { 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         { 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         { 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         { 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         { 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         { 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090910, 'color':'crimson' },]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter2.csv',index=False)


Voting:



,773.1,773.10,773.12,773.5,773.6,773.8,773.9,774.1,774.3,774.4,774.5
0,0.94662,0.94955,0.94354,0.95260,0.95319,0.95342,0.94988,0.95003,0.94204,0.94994,0.94936
1,0.96766,0.96982,0.96756,0.97237,0.97232,0.97231,0.97000,0.97046,0.96721,0.97036,0.96999
2,0.42957,0.41967,0.41526,0.40708,0.41239,0.41295,0.41985,0.41613,0.40008,0.41705,0.41794
3,0.92503,0.93274,0.91951,0.93721,0.93722,0.93765,0.93292,0.93221,0.92010,0.93196,0.93100
4,0.96728,0.96954,0.96719,0.97259,0.97251,0.97252,0.96976,0.97037,0.96706,0.97028,0.96986
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98263,0.98302,0.98597,0.98475,0.98427,0.98402,0.98333,0.98415,0.98643,0.98406,0.98399
254565,0.86085,0.87907,0.85124,0.87483,0.87538,0.87637,0.87944,0.87258,0.85077,0.87135,0.87065
254566,0.94854,0.95367,0.94687,0.95661,0.95651,0.95668,0.95395,0.95364,0.94694,0.95339,0.95281
254567,0.97733,0.97780,0.97969,0.97990,0.97964,0.97949,0.97807,0.97896,0.97946,0.97889,0.97874



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94741,0.00262
1,593995,0.97046,0.96902,0.00144
2,593996,0.41613,0.41179,0.00434
3,593997,0.93221,0.92774,0.00447
4,593998,0.97037,0.96879,0.00158
...,...,...,...,...
254564,848558,0.98415,0.98363,0.00052
254565,848559,0.87258,0.86501,0.00757
254566,848560,0.95364,0.95098,0.00267
254567,848561,0.97896,0.97849,0.00047


905.3641624762954 0.0


## Iteration_2

In [4]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         { 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         { 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         { 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         { 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         { 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter3.csv',index=False)


Voting:



,773.10,773.12,773.5,773.6,773.8,773.9,774.1,774.3,774.4,774.5,iter2
0,0.94955,0.94354,0.95260,0.95319,0.95342,0.94988,0.95003,0.94204,0.94994,0.94936,0.94741
1,0.96982,0.96756,0.97237,0.97232,0.97231,0.97000,0.97046,0.96721,0.97036,0.96999,0.96902
2,0.41967,0.41526,0.40708,0.41239,0.41295,0.41985,0.41613,0.40008,0.41705,0.41794,0.41179
3,0.93274,0.91951,0.93721,0.93722,0.93765,0.93292,0.93221,0.92010,0.93196,0.93100,0.92774
4,0.96954,0.96719,0.97259,0.97251,0.97252,0.96976,0.97037,0.96706,0.97028,0.96986,0.96879
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98302,0.98597,0.98475,0.98427,0.98402,0.98333,0.98415,0.98643,0.98406,0.98399,0.98363
254565,0.87907,0.85124,0.87483,0.87538,0.87637,0.87944,0.87258,0.85077,0.87135,0.87065,0.86501
254566,0.95367,0.94687,0.95661,0.95651,0.95668,0.95395,0.95364,0.94694,0.95339,0.95281,0.95098
254567,0.97780,0.97969,0.97990,0.97964,0.97949,0.97807,0.97896,0.97946,0.97889,0.97874,0.97849



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94752,0.00251
1,593995,0.97046,0.96920,0.00126
2,593996,0.41613,0.41108,0.00505
3,593997,0.93221,0.92811,0.00410
4,593998,0.97037,0.96900,0.00137
...,...,...,...,...
254564,848558,0.98415,0.98377,0.00039
254565,848559,0.87258,0.86557,0.00701
254566,848560,0.95364,0.95131,0.00234
254567,848561,0.97896,0.97865,0.00031


860.6396315632119 0.0


## Iteration_3

In [5]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         { 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         { 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         #{ 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         { 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         { 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
         { 'name': f'iter3', 'weight':+0.090909090910, 'color':'darkmagenta' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter4.csv',index=False)


Voting:



,773.10,773.12,773.5,773.8,773.9,774.1,774.3,774.4,774.5,iter2,iter3
0,0.94955,0.94354,0.95260,0.95342,0.94988,0.95003,0.94204,0.94994,0.94936,0.94741,0.94752
1,0.96982,0.96756,0.97237,0.97231,0.97000,0.97046,0.96721,0.97036,0.96999,0.96902,0.96920
2,0.41967,0.41526,0.40708,0.41295,0.41985,0.41613,0.40008,0.41705,0.41794,0.41179,0.41108
3,0.93274,0.91951,0.93721,0.93765,0.93292,0.93221,0.92010,0.93196,0.93100,0.92774,0.92811
4,0.96954,0.96719,0.97259,0.97252,0.96976,0.97037,0.96706,0.97028,0.96986,0.96879,0.96900
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98302,0.98597,0.98475,0.98402,0.98333,0.98415,0.98643,0.98406,0.98399,0.98363,0.98377
254565,0.87907,0.85124,0.87483,0.87637,0.87944,0.87258,0.85077,0.87135,0.87065,0.86501,0.86557
254566,0.95367,0.94687,0.95661,0.95668,0.95395,0.95364,0.94694,0.95339,0.95281,0.95098,0.95131
254567,0.97780,0.97969,0.97990,0.97949,0.97807,0.97896,0.97946,0.97889,0.97874,0.97849,0.97865



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94719,0.00284
1,593995,0.97046,0.96905,0.00141
2,593996,0.41613,0.41090,0.00523
3,593997,0.93221,0.92748,0.00473
4,593998,0.97037,0.96882,0.00155
...,...,...,...,...
254564,848558,0.98415,0.98371,0.00044
254565,848559,0.87258,0.86432,0.00826
254566,848560,0.95364,0.95099,0.00266
254567,848561,0.97896,0.97854,0.00042


977.0699639299071 0.0


## Iteration_4

In [6]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         { 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         { 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         #{ 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         { 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         #{ 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
         { 'name': f'iter3', 'weight':+0.090909090909, 'color':'darkmagenta' },
         { 'name': f'iter4', 'weight':+0.090909090910, 'color':'maroon' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter5.csv',index=False)


Voting:



,773.10,773.12,773.5,773.8,774.1,774.3,774.4,774.5,iter2,iter3,iter4
0,0.94955,0.94354,0.95260,0.95342,0.95003,0.94204,0.94994,0.94936,0.94741,0.94752,0.94719
1,0.96982,0.96756,0.97237,0.97231,0.97046,0.96721,0.97036,0.96999,0.96902,0.96920,0.96905
2,0.41967,0.41526,0.40708,0.41295,0.41613,0.40008,0.41705,0.41794,0.41179,0.41108,0.41090
3,0.93274,0.91951,0.93721,0.93765,0.93221,0.92010,0.93196,0.93100,0.92774,0.92811,0.92748
4,0.96954,0.96719,0.97259,0.97252,0.97037,0.96706,0.97028,0.96986,0.96879,0.96900,0.96882
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98302,0.98597,0.98475,0.98402,0.98415,0.98643,0.98406,0.98399,0.98363,0.98377,0.98371
254565,0.87907,0.85124,0.87483,0.87637,0.87258,0.85077,0.87135,0.87065,0.86501,0.86557,0.86432
254566,0.95367,0.94687,0.95661,0.95668,0.95364,0.94694,0.95339,0.95281,0.95098,0.95131,0.95099
254567,0.97780,0.97969,0.97990,0.97949,0.97896,0.97946,0.97889,0.97874,0.97849,0.97865,0.97854



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94683,0.00320
1,593995,0.97046,0.96892,0.00154
2,593996,0.41613,0.41019,0.00594
3,593997,0.93221,0.92684,0.00537
4,593998,0.97037,0.96870,0.00168
...,...,...,...,...
254564,848558,0.98415,0.98376,0.00039
254565,848559,0.87258,0.86320,0.00939
254566,848560,0.95364,0.95063,0.00302
254567,848561,0.97896,0.97860,0.00036


1055.5590788900129 0.0


## Iteration_5

In [7]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         { 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         #{ 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         #{ 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         { 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         #{ 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
         { 'name': f'iter3', 'weight':+0.090909090909, 'color':'darkmagenta' },
         { 'name': f'iter4', 'weight':+0.090909090909, 'color':'maroon' },
         { 'name': f'iter5', 'weight':+0.090909090910, 'color':'deeppink' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter6.csv',index=False)


Voting:



,773.10,773.12,773.8,774.1,774.3,774.4,774.5,iter2,iter3,iter4,iter5
0,0.94955,0.94354,0.95342,0.95003,0.94204,0.94994,0.94936,0.94741,0.94752,0.94719,0.94683
1,0.96982,0.96756,0.97231,0.97046,0.96721,0.97036,0.96999,0.96902,0.96920,0.96905,0.96892
2,0.41967,0.41526,0.41295,0.41613,0.40008,0.41705,0.41794,0.41179,0.41108,0.41090,0.41019
3,0.93274,0.91951,0.93765,0.93221,0.92010,0.93196,0.93100,0.92774,0.92811,0.92748,0.92684
4,0.96954,0.96719,0.97252,0.97037,0.96706,0.97028,0.96986,0.96879,0.96900,0.96882,0.96870
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98302,0.98597,0.98402,0.98415,0.98643,0.98406,0.98399,0.98363,0.98377,0.98371,0.98376
254565,0.87907,0.85124,0.87637,0.87258,0.85077,0.87135,0.87065,0.86501,0.86557,0.86432,0.86320
254566,0.95367,0.94687,0.95668,0.95364,0.94694,0.95339,0.95281,0.95098,0.95131,0.95099,0.95063
254567,0.97780,0.97969,0.97949,0.97896,0.97946,0.97889,0.97874,0.97849,0.97865,0.97854,0.97860



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94646,0.00357
1,593995,0.97046,0.96877,0.00169
2,593996,0.41613,0.41061,0.00552
3,593997,0.93221,0.92614,0.00606
4,593998,0.97037,0.96854,0.00183
...,...,...,...,...
254564,848558,0.98415,0.98372,0.00043
254565,848559,0.87258,0.86209,0.01049
254566,848560,0.95364,0.95025,0.00339
254567,848561,0.97896,0.97855,0.00041


1165.1866112856294 0.0


## Iteration_6

In [8]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         { 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         #{ 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         #{ 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         #{ 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         #{ 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
         { 'name': f'iter3', 'weight':+0.090909090909, 'color':'darkmagenta' },
         { 'name': f'iter4', 'weight':+0.090909090909, 'color':'maroon' },
         { 'name': f'iter5', 'weight':+0.090909090909, 'color':'deeppink' },
         { 'name': f'iter6', 'weight':+0.090909090910, 'color':'dimgray' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter7.csv',index=False)


Voting:



,773.10,773.12,774.1,774.3,774.4,774.5,iter2,iter3,iter4,iter5,iter6
0,0.94955,0.94354,0.95003,0.94204,0.94994,0.94936,0.94741,0.94752,0.94719,0.94683,0.94646
1,0.96982,0.96756,0.97046,0.96721,0.97036,0.96999,0.96902,0.96920,0.96905,0.96892,0.96877
2,0.41967,0.41526,0.41613,0.40008,0.41705,0.41794,0.41179,0.41108,0.41090,0.41019,0.41061
3,0.93274,0.91951,0.93221,0.92010,0.93196,0.93100,0.92774,0.92811,0.92748,0.92684,0.92614
4,0.96954,0.96719,0.97037,0.96706,0.97028,0.96986,0.96879,0.96900,0.96882,0.96870,0.96854
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98302,0.98597,0.98415,0.98643,0.98406,0.98399,0.98363,0.98377,0.98371,0.98376,0.98372
254565,0.87907,0.85124,0.87258,0.85077,0.87135,0.87065,0.86501,0.86557,0.86432,0.86320,0.86209
254566,0.95367,0.94687,0.95364,0.94694,0.95339,0.95281,0.95098,0.95131,0.95099,0.95063,0.95025
254567,0.97780,0.97969,0.97896,0.97946,0.97889,0.97874,0.97849,0.97865,0.97854,0.97860,0.97855



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94606,0.00397
1,593995,0.97046,0.96863,0.00183
2,593996,0.41613,0.41029,0.00583
3,593997,0.93221,0.92549,0.00672
4,593998,0.97037,0.96840,0.00197
...,...,...,...,...
254564,848558,0.98415,0.98368,0.00047
254565,848559,0.87258,0.86093,0.01166
254566,848560,0.95364,0.94990,0.00374
254567,848561,0.97896,0.97851,0.00046


1277.0041547945798 0.0


## Iteration_7

In [9]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         { 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         #{ 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         #{ 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         #{ 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         #{ 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         #{ 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
         { 'name': f'iter3', 'weight':+0.090909090909, 'color':'darkmagenta' },
         { 'name': f'iter4', 'weight':+0.090909090909, 'color':'maroon' },
         { 'name': f'iter5', 'weight':+0.090909090909, 'color':'deeppink' },
         { 'name': f'iter6', 'weight':+0.090909090909, 'color':'dimgray' },
         { 'name': f'iter7', 'weight':+0.090909090910, 'color':'magenta' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);  df.to_csv(new_path +'/iter8.csv',index=False)


Voting:



,773.10,774.1,774.3,774.4,774.5,iter2,iter3,iter4,iter5,iter6,iter7
0,0.94955,0.95003,0.94204,0.94994,0.94936,0.94741,0.94752,0.94719,0.94683,0.94646,0.94606
1,0.96982,0.97046,0.96721,0.97036,0.96999,0.96902,0.96920,0.96905,0.96892,0.96877,0.96863
2,0.41967,0.41613,0.40008,0.41705,0.41794,0.41179,0.41108,0.41090,0.41019,0.41061,0.41029
3,0.93274,0.93221,0.92010,0.93196,0.93100,0.92774,0.92811,0.92748,0.92684,0.92614,0.92549
4,0.96954,0.97037,0.96706,0.97028,0.96986,0.96879,0.96900,0.96882,0.96870,0.96854,0.96840
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98302,0.98415,0.98643,0.98406,0.98399,0.98363,0.98377,0.98371,0.98376,0.98372,0.98368
254565,0.87907,0.87258,0.85077,0.87135,0.87065,0.86501,0.86557,0.86432,0.86320,0.86209,0.86093
254566,0.95367,0.95364,0.94694,0.95339,0.95281,0.95098,0.95131,0.95099,0.95063,0.95025,0.94990
254567,0.97780,0.97896,0.97946,0.97889,0.97874,0.97849,0.97865,0.97854,0.97860,0.97855,0.97851



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94641,0.00362
1,593995,0.97046,0.96878,0.00169
2,593996,0.41613,0.40962,0.00651
3,593997,0.93221,0.92630,0.00591
4,593998,0.97037,0.96857,0.00180
...,...,...,...,...
254564,848558,0.98415,0.98364,0.00051
254565,848559,0.87258,0.86224,0.01034
254566,848560,0.95364,0.95032,0.00333
254567,848561,0.97896,0.97847,0.00049


1177.1302395985938 0.0


## Iteration_8

In [10]:
params = {
      'path'     : new_path+"/",            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc',0.30,0.70 ],
      'subwts'   : [e/200 for e in [+5,+4,+3,+2,+1, 0, -1,-2,-3,-4,-5]],
      'subm'     : [
         #{ 'name': f'773.1', 'weight':+0.090909090909, 'color':'green' },
         #{ 'name': f'773.10','weight':+0.090909090909, 'color':'seagreen' },
         #{ 'name': f'773.12','weight':+0.090909090909, 'color':'olivedrab' },
         #{ 'name': f'773.5', 'weight':+0.090909090909, 'color':'darkgreen' },
         #{ 'name': f'773.6', 'weight':+0.090909090909, 'color':'forestgreen' },
         #{ 'name': f'773.8', 'weight':+0.090909090909, 'color':'limegreen' }, 
         #{ 'name': f'773.9', 'weight':+0.090909090909, 'color':'lime' },
         { 'name': f'774.1', 'weight':+0.090909090909, 'color':'tomato' },
         { 'name': f'774.3', 'weight':+0.090909090909, 'color':'orangered' },
         { 'name': f'774.4', 'weight':+0.090909090909, 'color':'red' },
         { 'name': f'774.5', 'weight':+0.090909090909, 'color':'crimson' },
         { 'name': f'iter2', 'weight':+0.090909090909, 'color':'mediumblue' },
         { 'name': f'iter3', 'weight':+0.090909090909, 'color':'darkmagenta' },
         { 'name': f'iter4', 'weight':+0.090909090909, 'color':'maroon' },
         { 'name': f'iter5', 'weight':+0.090909090909, 'color':'deeppink' },
         { 'name': f'iter6', 'weight':+0.090909090909, 'color':'dimgray' },
         { 'name': f'iter7', 'weight':+0.090909090909, 'color':'magenta' },
         { 'name': f'iter8', 'weight':+0.090909090910, 'color':'darkorchid' },
      ]
}
h_blend(params, fig1=True)

df = voting(rem_left=0,rem_right=4);


Voting:



,774.1,774.3,774.4,774.5,iter2,iter3,iter4,iter5,iter6,iter7,iter8
0,0.95003,0.94204,0.94994,0.94936,0.94741,0.94752,0.94719,0.94683,0.94646,0.94606,0.94641
1,0.97046,0.96721,0.97036,0.96999,0.96902,0.96920,0.96905,0.96892,0.96877,0.96863,0.96878
2,0.41613,0.40008,0.41705,0.41794,0.41179,0.41108,0.41090,0.41019,0.41061,0.41029,0.40962
3,0.93221,0.92010,0.93196,0.93100,0.92774,0.92811,0.92748,0.92684,0.92614,0.92549,0.92630
4,0.97037,0.96706,0.97028,0.96986,0.96879,0.96900,0.96882,0.96870,0.96854,0.96840,0.96857
...,...,...,...,...,...,...,...,...,...,...,...
254564,0.98415,0.98643,0.98406,0.98399,0.98363,0.98377,0.98371,0.98376,0.98372,0.98368,0.98364
254565,0.87258,0.85077,0.87135,0.87065,0.86501,0.86557,0.86432,0.86320,0.86209,0.86093,0.86224
254566,0.95364,0.94694,0.95339,0.95281,0.95098,0.95131,0.95099,0.95063,0.95025,0.94990,0.95032
254567,0.97896,0.97946,0.97889,0.97874,0.97849,0.97865,0.97854,0.97860,0.97855,0.97851,0.97847



Comparison: df_Top _x  vs  df_vote _y



,id,loan_paid_back_x,loan_paid_back_y,delta_back's
0,593994,0.95003,0.94626,0.00377
1,593995,0.97046,0.96872,0.00175
2,593996,0.41613,0.40932,0.00680
3,593997,0.93221,0.92605,0.00615
4,593998,0.97037,0.96851,0.00186
...,...,...,...,...
254564,848558,0.98415,0.98373,0.00043
254565,848559,0.87258,0.86179,0.01079
254566,848560,0.95364,0.95018,0.00346
254567,848561,0.97896,0.97856,0.00040


1200.920576832239 0.0


In [11]:
if os.path.isdir(new_path): shutil.rmtree(new_path)

## Submit

In [12]:
df.to_csv('submission.csv',index=False)
df

,id,loan_paid_back
0,593994,0.94626
1,593995,0.96872
2,593996,0.40932
3,593997,0.92605
4,593998,0.96851
...,...,...
254564,848558,0.98373
254565,848559,0.86179
254566,848560,0.95018
254567,848561,0.97856
